# Integrated Air Quality → Health Notebook

This notebook is an end-to-end pipeline for the NASA Space Apps hackathon: it ingests TEMPO / OpenAQ pollutant CSVs, weather CSVs, and NLDAS netCDF (if available), creates region centroids, synthesizes health labels, trains pollutant forecast and health-impact models, and exports GeoJSON suitable for a frontend map.

**Usage:** edit the configuration cell below to point to your files, then run cells from top to bottom. Cells include explanation and safety notes; where you lack real health data the notebook will generate synthetic labels so you can demo the full pipeline.


In [ ]:
# ===== CONFIG: set file paths and parameters =====
ROOT = '/mnt/data'
TEMPO_NO2_CSV = 'tempo_no2.csv'
TEMPO_HCHO_CSV = 'tempo_hcho.csv'
OPENAQ_CSV = 'openaq.csv'
WEATHER_CSV = 'weatherdatamap.csv'
NLDAS_FILE = 'NLDAS_FORA0125_H.A20250928.1200.020.nc'
REGIONS_CSV = 'regions.csv'
HEALTH_CSV = 'health_counts.csv'
AGGREGATION = 'D'
ROLL_WINDOWS = [1,3,6,24]
LABEL_Z = 1.5
TARGET_POLLUTANTS = ['no2','hcho']
NUM_REGIONS_SAMPLE = 200
print('CONFIG set. Edit paths above if needed.')


In [ ]:
import os, json, warnings
from pathlib import Path
import numpy as np, pandas as pd
import xarray as xr
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_recall_fscore_support, roc_auc_score
import joblib
print('Libraries imported')


## 1) Load available data

The notebook will attempt to load CSVs and the NLDAS NetCDF. If files are missing it will skip those steps but continue.

In [ ]:
def safe_read_csv(path, parse_time=True, time_col_candidates=['time','datetime','date']):
    p = Path(path)
    if not p.exists():
        print(f'File not found: {path} - skipping')
        return None
    df = pd.read_csv(p)
    tcol = None
    for c in time_col_candidates:
        if c in df.columns:
            tcol = c; break
    if parse_time and tcol is not None:
        df[tcol] = pd.to_datetime(df[tcol])
        df = df.rename(columns={tcol:'time'})
    return df

data = {}
data['tempo_no2'] = safe_read_csv(TEMPO_NO2_CSV)
data['tempo_hcho'] = safe_read_csv(TEMPO_HCHO_CSV)
data['openaq'] = safe_read_csv(OPENAQ_CSV)
data['weather'] = safe_read_csv(WEATHER_CSV)
data['health'] = safe_read_csv(HEALTH_CSV)
if Path(REGIONS_CSV).exists():
    data['regions'] = pd.read_csv(REGIONS_CSV)
    print('Loaded regions.csv with', len(data['regions']), 'regions')
else:
    data['regions'] = None
    print('regions.csv not found; will attempt to generate from NLDAS if available')

if Path(NLDAS_FILE).exists():
    print('Opening NLDAS file:', NLDAS_FILE)
    ds_nldas = xr.open_dataset(NLDAS_FILE)
    print('NLDAS variables:', list(ds_nldas.data_vars))
else:
    ds_nldas = None
    print('NLDAS file not found; NLDAS steps will be skipped')


## 2) Create regions (if missing)

If `regions.csv` is not provided, create `regions_from_nldas.csv` by sampling grid cell centers from NLDAS or by restricting to a bounding box. This provides centroids for aggregation and demo mapping.

In [ ]:
if data.get('regions') is None:
    if ds_nldas is None:
        print('Cannot create regions: no regions.csv and no NLDAS file. Provide regions.csv or NLDAS.')
    else:
        print('Generating regions_from_nldas.csv from NLDAS grid...')
        lats = ds_nldas['lat'].values
        lons = ds_nldas['lon'].values
        lon2d, lat2d = np.meshgrid(lons, lats)
        lat_flat = lat2d.ravel(); lon_flat = lon2d.ravel()
        N = min(NUM_REGIONS_SAMPLE, len(lat_flat))
        idxs = np.linspace(0, len(lat_flat)-1, N, dtype=int)
        regions_df = pd.DataFrame({'region_id':[f'R{i+1}' for i in range(N)], 'lat':lat_flat[idxs], 'lon':lon_flat[idxs]})
        regions_df.to_csv('regions_from_nldas.csv', index=False)
        data['regions'] = regions_df
        print('Wrote regions_from_nldas.csv with', len(regions_df), 'regions')
else:
    print('Using provided regions.csv')


## 3) Standardize missing values and basic cleaning

Replace sentinel -9999 with NaN, inspect null percentages, and for station-like CSVs perform per-location interpolation. For NLDAS, perform temporal interpolation along time dimension (conservative).

In [ ]:
for k,v in list(data.items()):
    if isinstance(v, pd.DataFrame):
        data[k] = v.replace(-9999, np.nan)
        print(k, 'loaded with shape', data[k].shape, 'nulls per col:\n', data[k].isnull().sum().to_dict())

if ds_nldas is not None:
    try:
        ds_nldas = ds_nldas.interpolate_na(dim='time', method='linear')
        ds_nldas = ds_nldas.ffill(dim='time').bfill(dim='time')
        print('NLDAS interpolation done')
    except Exception as e:
        print('NLDAS interpolation failed or too large:', e)


## 4) Map pollutant/observation points to regions and aggregate

We take TEMP/OPENAQ point-level data and map to nearest region centroid (region_id), then aggregate (mean) to daily/hourly depending on AGGREGATION.

In [ ]:
def nearest_region(df_points, regions_df, lat_col='lat', lon_col='lon'):
    pts = df_points.copy()
    # compute distances to all regions (brute force) and pick nearest
    regs = regions_df[[lat_col, lon_col, 'region_id']].values
    out_region = []
    for i,row in pts.iterrows():
        latp = row[lat_col]; lonp = row[lon_col]
        d2 = (regions_df[lat_col]-latp)**2 + (regions_df[lon_col]-lonp)**2
        out_region.append(regions_df.loc[d2.idxmin(), 'region_id'])
    pts['region_id'] = out_region
    return pts

pts = []
for src in ['tempo_no2','tempo_hcho','openaq']:
    df = data.get(src)
    if isinstance(df, pd.DataFrame) and 'time' in df.columns and 'lat' in df.columns and 'lon' in df.columns:
        pts.append(df)

if len(pts)==0 and ds_nldas is None:
    print('No pollutant or NLDAS data available to proceed. Please provide at least one.')
else:
    if len(pts)>0:
        all_pts = pd.concat(pts, ignore_index=True, sort=False)
        print('Combined point data shape', all_pts.shape)
        regions_df = data['regions']
        pts_mapped = nearest_region(all_pts, regions_df)
        pts_mapped['time'] = pd.to_datetime(pts_mapped['time'])
        if AGGREGATION=='D':
            pts_mapped['date'] = pts_mapped['time'].dt.floor('D')
            agg = pts_mapped.groupby(['region_id','date']).mean().reset_index()
        else:
            pts_mapped['hour'] = pts_mapped['time'].dt.floor('H')
            agg = pts_mapped.groupby(['region_id','hour']).mean().reset_index()
        data['pollutant_regions'] = agg
        print('Aggregated pollutant data to regions:', agg.shape)
    if ds_nldas is not None:
        regions_df = data['regions']
        recs = []
        times = pd.to_datetime(ds_nldas['time'].values)
        for rid, r in regions_df.iterrows():
            lat_idx = int(np.abs(ds_nldas['lat'].values - r['lat']).argmin())
            lon_idx = int(np.abs(ds_nldas['lon'].values - r['lon']).argmin())
            tmp = pd.DataFrame({'time': times})
            for v in ['Tair','Qair','PSurf','Wind_E','Wind_N','Rainf','CRainf_frac','SWdown','PotEvap','CAPE']:
                if v in ds_nldas:
                    tmp[v] = ds_nldas[v][:, lat_idx, lon_idx].values
            tmp['region_id'] = r['region_id']; tmp['lat']=r['lat']; tmp['lon']=r['lon']
            recs.append(tmp)
        reg_all = pd.concat(recs, ignore_index=True)
        if AGGREGATION=='D':
            reg_all['date'] = pd.to_datetime(reg_all['time']).dt.floor('D')
            reg_daily = reg_all.groupby(['region_id','date']).mean().reset_index()
            data['nldas_regions'] = reg_daily
            print('Extracted NLDAS -> regions daily shape', reg_daily.shape)
        else:
            data['nldas_regions'] = reg_all
            print('Extracted NLDAS -> regions hourly shape', reg_all.shape)


## 5) Clean, fill small gaps, and engineer features

Compute wind speed/direction, rolling rainfall windows, convective-weighted intensity, heat index if possible, and lag features for pollutants.

In [ ]:
def add_features_regions(df):
    df = df.copy()
    if 'Wind_E' in df.columns and 'Wind_N' in df.columns:
        df['wind_speed'] = np.sqrt(df['Wind_E']**2 + df['Wind_N']**2)
        df['wind_dir_to'] = (np.degrees(np.arctan2(df['Wind_E'], df['Wind_N'])) % 360)
    if 'Rainf' in df.columns:
        df = df.sort_values('date' if 'date' in df.columns else 'time')
        df['rain_roll_3'] = df.groupby('region_id')['Rainf'].rolling(window=3, min_periods=1).sum().reset_index(0,drop=True)
        if 'CRainf_frac' in df.columns:
            df['conv_intensity_3'] = df['rain_roll_3'] * df['CRainf_frac']
    if all(c in df.columns for c in ['Tair','Qair','PSurf']):
        T_K = df['Tair']; T_C = T_K - 273.15
        es = 6.112 * np.exp((17.67 * T_C) / (T_C + 243.5))
        p_hpa = df['PSurf'] / 100.0
        e = df['Qair'] * p_hpa / (0.622 + 0.378*df['Qair'] + 1e-12)
        df['RH'] = np.clip((e / es) * 100.0, 0, 100)
        T_F = (T_C * 9/5) + 32
        RH = df['RH']
        HI = -42.379 + 2.04901523*T_F + 10.14333127*RH - 0.22475541*T_F*RH - 6.83783e-3*T_F**2 - 5.481717e-2*RH**2 + 1.22874e-3*T_F**2*RH + 8.5282e-4*T_F*RH**2 - 1.99e-6*T_F**2*RH**2
        df['heat_index_C'] = (HI - 32) * 5/9
    return df

regions_table = None
if data.get('nldas_regions') is not None:
    regions_table = data['nldas_regions']
    if 'date' not in regions_table.columns and 'time' in regions_table.columns:
        regions_table['date'] = pd.to_datetime(regions_table['time']).dt.floor('D')
    regions_table = add_features_regions(regions_table)
    data['regions_features'] = regions_table
elif data.get('pollutant_regions') is not None:
    regions_table = data['pollutant_regions']
    if 'date' not in regions_table.columns and 'time' in regions_table.columns:
        regions_table['date'] = pd.to_datetime(regions_table['time']).dt.floor('D')
    regions_table = add_features_regions(regions_table)
    data['regions_features'] = regions_table
else:
    print('No region-level table available to engineer features')
    data['regions_features'] = None

if data['regions_features'] is not None:
    print('Regions features shape:', data['regions_features'].shape)


## 6) Create synthetic health labels (two strategies)

Strategy A: pollutant z-score per region. Strategy B: compound event (pollutant > percentile AND low wind). Notebook produces both and combines them.

In [ ]:
def synth_labels_zscore(df, pollutant='no2', z_thresh=1.5):
    df = df.copy()
    base = df.groupby('region_id')[pollutant].agg(['mean','std']).reset_index().rename(columns={'mean':'bmean','std':'bstd'})
    df = df.merge(base, on='region_id', how='left')
    df['z'] = (df[pollutant] - df['bmean']) / (df['bstd'].replace(0,np.nan)+1e-9)
    df['label_z'] = (df['z'] > z_thresh).astype(int)
    return df

def synth_labels_compound(df, pollutant='no2', pct=0.9, wind_thresh=2.0):
    df = df.copy()
    pr = df.groupby('region_id')[pollutant].transform(lambda x: x.quantile(pct))
    df['label_compound'] = (((df[pollutant] >= pr) & (df.get('wind_speed', 0) < wind_thresh)).astype(int))
    return df

if data.get('regions_features') is not None:
    rf = data['regions_features'].copy()
    pollutant_col = 'no2' if 'no2' in rf.columns else ('hcho' if 'hcho' in rf.columns else None)
    if pollutant_col is None:
        print('No pollutant column found in regions_features; creating synthetic pollutant using random noise for demo')
        np.random.seed(0)
        rf['no2'] = np.random.gamma(shape=2.0, scale=10.0, size=len(rf))
        pollutant_col = 'no2'
    rf = synth_labels_zscore(rf, pollutant=pollutant_col, z_thresh=LABEL_Z)
    rf = synth_labels_compound(rf, pollutant=pollutant_col)
    rf['label'] = ((rf['label_z'] + rf['label_compound']) > 0).astype(int)
    data['regions_labeled'] = rf
    print('Synthetic labels created; sample:')
    cols_show = [c for c in ['region_id','date', pollutant_col, 'z','label_z','label_compound','label'] if c in rf.columns]
    print(rf[cols_show].head())
else:
    print('No regions_features available, cannot create synthetic labels')


## 7) Prepare model dataset: lag features and train/test split

We create lagged pollutant features (t-1, t-2, etc.) and then split time-aware for training.

In [ ]:
def prepare_model_df(rf, pollutant='no2', lags=[1,2,3]):
    df = rf.copy().sort_values(['region_id','date'])
    for lag in lags:
        df[f'{pollutant}_lag{lag}'] = df.groupby('region_id')[pollutant].shift(lag)
    df['label_next'] = df.groupby('region_id')['label'].shift(-1)
    df = df.dropna(subset=[f'{pollutant}_lag1', 'label_next'])
    return df

model_df = None
if data.get('regions_labeled') is not None:
    model_df = prepare_model_df(data['regions_labeled'], 'no2', lags=[1,2,3])
    print('Model DF shape:', model_df.shape)
else:
    print('No labeled regions to prepare model DF')


## 8) Train models

Two models: pollutant forecast (regression) and health-impact classifier (binary). We use LightGBM for both for speed and explainability.

In [ ]:
models = {}
if model_df is not None and len(model_df)>0:
    feat_pollutant = [c for c in model_df.columns if 'lag' in c or c in ['Tair','wind_speed','rain_roll_3']]
    target_poll = 'no2'
    split = int(0.8 * len(model_df))
    X = model_df[feat_pollutant].fillna(0)
    y = model_df[target_poll]
    dtrain = lgb.Dataset(X.iloc[:split], label=y.iloc[:split])
    params_reg = {'objective':'regression','metric':'rmse','verbosity':-1}
    m_reg = lgb.train(params_reg, dtrain, num_boost_round=200)
    ypred = m_reg.predict(X.iloc[split:])
    print('Pollutant reg RMSE:', np.sqrt(mean_squared_error(y.iloc[split:], ypred)))
    models['pollutant_reg'] = m_reg
    joblib.dump(m_reg, 'pollutant_reg.joblib')

    feat_health = feat_pollutant.copy()
    Xh = model_df[feat_health].fillna(0)
    yh = model_df['label_next'].astype(int)
    dtrain_h = lgb.Dataset(Xh.iloc[:split], label=yh.iloc[:split])
    params_clf = {'objective':'binary','metric':'auc','verbosity':-1}
    m_clf = lgb.train(params_clf, dtrain_h, num_boost_round=200)
    yprob = m_clf.predict(Xh.iloc[split:])
    yhat = (yprob>=0.5).astype(int)
    p,r,f,_ = precision_recall_fscore_support(yh.iloc[split:], yhat, average='binary', zero_division=0)
    auc = roc_auc_score(yh.iloc[split:], yprob)
    print('Health classifier P,R,F1,AUC:', p, r, f, auc)
    models['health_clf'] = m_clf
    joblib.dump(m_clf, 'health_clf.joblib')
else:
    print('No training data available; ensure regions and pollutant columns exist')


## 9) Produce GeoJSON predictions for the latest date

Use trained health classifier to predict probability for latest date per region and export GeoJSON with advice categories.

In [ ]:
def preds_to_geojson(model_clf, model_df, regions_df, features, out_path='predictions_last_day.geojson'):
    latest = model_df.groupby('region_id').apply(lambda g: g.sort_values('date').iloc[-1]).reset_index(drop=True)
    Xpred = latest[features].fillna(0)
    probs = model_clf.predict(Xpred)
    latest['pred_prob'] = probs
    feats = []
    for _, r in latest.iterrows():
        reg = regions_df[regions_df['region_id']==r['region_id']].iloc[0]
        lat = float(reg['lat']); lon = float(reg['lon'])
        p = float(r['pred_prob'])
        cat = 'Low' if p<0.2 else ('Moderate' if p<0.5 else 'High')
        advice = 'No action' if cat=='Low' else ('Sensitive groups reduce outdoor activity' if cat=='Moderate' else 'Avoid outdoor activity; clinics on alert')
        feats.append({'type':'Feature','geometry':{'type':'Point','coordinates':[lon, lat]},'properties':{'region_id':r['region_id'],'pred_prob':p,'category':cat,'advice':advice}})
    geo = {'type':'FeatureCollection','features':feats}
    with open(out_path,'w') as f:
        json.dump(geo, f)
    print('Wrote', out_path)
    return out_path

if 'health_clf' in models and model_df is not None:
    features_used = feat_health
    out = preds_to_geojson(models['health_clf'], model_df, data['regions'], features_used)
else:
    print('No health classifier available to produce predictions')


## Done

Files produced (if steps ran):
- regions_from_nldas.csv (if created)
- regions_features table (in memory)
- regions_labeled (in memory)
- pollutant_reg.joblib, health_clf.joblib (saved models if training ran)
- predictions_last_day.geojson (if model trained)

Next: build FastAPI to serve predictions or React/Leaflet frontend to visualize geojson. Remember to clearly state synthetic labels if using them for demo.